In [ ]:
#@title نصب کتابخانه ها
import base64

encoded_url = "aHR0cHM6Ly9naXRodWIuY29tL3lhcmFuYmFyemkvYWlnb2xkZW4tYXVkaW8tdG8tdGV4dC5naXQ="
decoded_url = base64.b64decode(encoded_url.encode()).decode()
!pip install git+$decoded_url
! pip install edge_tts
!pip install yt-dlp
!pip install pysrt
!pip install rubberband-cli
!pip install pydub
!sudo apt update && sudo apt install ffmpeg

In [ ]:
#@title اتصال به گوگل درایو
#@markdown #### فقط در صورتی اجرا کنید که ویدیوی مورد نظر در گوگل درایو باشد
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title آپلود ویدیو از سه طریق مختلف
#@markdown ### `در صورتی که دانلود از یوتیوب ناموفق بود کوکی های مرورگر را قبل از اجرا پاک کنید`
from google.colab import files
from IPython.display import display
import ipywidgets as widgets
import yt_dlp
import os
import shutil
import glob

# پاکسازی فایل‌های قبلی
for file in glob.glob('input_video*'):
    os.remove(file)
    print(f"فایل {file} حذف شد.")

if os.path.exists('audio.wav'):
    os.remove('audio.wav')
    print("فایل صوتی قبلی حذف شد.")
if os.path.exists('audio.srt'):
    os.remove('audio.srt')
    print("فایل زیرنویس قبلی حذف شد.")
if os.path.exists('dubbing_project'):
    shutil.rmtree('dubbing_project')
    print("پوشه پروژه دوبله قبلی حذف شد.")

upload_method = "یوتیوب" #@param ["حافظه داخلی", "یوتیوب", "گوگل درایو"]
#@markdown #### محل وارد کردن لینک ویدیوی یوتیوب
YT_Link = "" #@param {type:"string"}
#@markdown #### انتخاب کیفیت ویدیوی یوتیوب
video_quality = "بالا" #@param ["بالا", "پایین"]
#@markdown #### محل وارد کردن مسیر گوگل درایو
drive_path = "" #@param {type:"string"}

def process_youtube(url, quality):
    if url.strip():
        # حذف فایل‌های قبلی
        for file in glob.glob('input_video*'):
            if os.path.exists(file):
                os.remove(file)

        # تنظیم فرمت مناسب
        if quality == "بالا":
            format_option = 'bestvideo+bestaudio/best'
        else:  # پایین
            format_option = 'best'

        # دانلود ویدیو با نام خاص
        temp_filename = 'temp_video.%(ext)s'
        video_opts = {
            'format': format_option,
            'outtmpl': temp_filename,
            'nocheckcertificate': True,
            'ignoreerrors': False,
            'no_warnings': False,
            'quiet': False
        }

        try:
            with yt_dlp.YoutubeDL(video_opts) as ydl:
                info = ydl.extract_info(url, download=True)
                downloaded_file = ydl.prepare_filename(info)

            print(f"فایل دانلود شده: {downloaded_file}")

            # تغییر نام فایل به input_video.mp4
            if os.path.exists(downloaded_file):
                # استخراج پسوند فایل و حفظ آن
                _, file_extension = os.path.splitext(downloaded_file)

                # تغییر نام فایل با حفظ پسوند
                final_filename = 'input_video' + file_extension
                os.rename(downloaded_file, final_filename)

                print(f"نام فایل به {final_filename} تغییر یافت")

                # اگر پسوند mp4 نیست، تبدیل به mp4
                if file_extension.lower() != '.mp4':
                    os.system(f'ffmpeg -i "{final_filename}" -c copy input_video.mp4 -y')
                    if os.path.exists('input_video.mp4'):
                        os.remove(final_filename)
                        print(f"فایل به فرمت MP4 تبدیل شد")

                # استخراج صدا
                os.system('ffmpeg -i input_video.mp4 -vn audio.wav -y')
                print(f"صدا از ویدیو استخراج شد.")
                return True
            else:
                print("فایل دانلود شده یافت نشد.")
                return False
        except Exception as e:
            print(f"خطا در دانلود: {str(e)}")
            return False
    return False

if upload_method == "یوتیوب" and YT_Link.strip():
    print(f"در حال دانلود ویدیو با کیفیت {video_quality}...")
    success = process_youtube(YT_Link, video_quality)
    if success:
        print(f"ویدیو با کیفیت {video_quality} دانلود شد.")
    else:
        print("دانلود ناموفق بود.")
elif upload_method == "گوگل درایو" and drive_path.strip():
    os.system(f'cp "{drive_path}" input_video.mp4')
    os.system('ffmpeg -i input_video.mp4 -vn audio.wav')
    print("ویدیو و فایل صوتی از گوگل درایو کپی شدند.")
elif upload_method == "حافظه داخلی":
    print("لطفاً ویدیوی خود را آپلود کنید:")
    uploaded = files.upload()
    video_file = next(iter(uploaded.keys()))
    os.system(f'ffmpeg -i "{video_file}" -vn audio.wav')
    os.rename(video_file, 'input_video.mp4')
    print("ویدیو آپلود و فایل صوتی استخراج شد.")

if os.path.exists('input_video.mp4'):
    print("عملیات با موفقیت انجام شد!")
else:
    print("خطا در دانلود ویدیو. لطفاً لینک دیگری را امتحان کنید یا از روش دیگری برای آپلود استفاده کنید.")

In [ ]:
#@title استخراج متن از فایل صوتی
from google.colab import files
import os

extraction_method = "Whisper" #@param ["Whisper", "آپلود زیرنویس"]

if extraction_method == "Whisper":
    if os.path.exists('audio.wav'):
        !whisper "audio.wav" --model large --output_dir ./ --output_format srt
        os.rename("audio.srt", "audio.srt")  # اطمینان از نام فایل صحیح
    else:
        print("لطفاً ابتدا یک فایل صوتی آپلود کنید.")

else:
    print("لطفاً فایل زیرنویس خود را با فرمت .srt آپلود کنید:")
    uploaded = files.upload()
    subtitle_file = next(iter(uploaded.keys()))
    os.rename(subtitle_file, 'audio.srt')

In [ ]:
#@title ترجمه زیرنویس
import pysrt
import google.generativeai as genai
from tqdm.notebook import tqdm
import time
from tenacity import retry, stop_after_attempt, wait_exponential
from google.colab import files
import os

translation_method = "هوش مصنوعی" #@param ["هوش مصنوعی", "آپلود زیرنویس بصورت دستی"]
source_language = "English (EN)" #@param ["English (EN)", "Persian (FA)", "German (DE)", "French (FR)", "Italian (IT)", "Spanish (ES)", "Chinese (ZH)", "Korean (KO)", "Russian (RU)", "Arabic (AR)", "Japanese (JA)"]
target_language = "Persian (FA)" #@param ["Persian (FA)", "English (EN)", "German (DE)", "French (FR)", "Italian (IT)", "Spanish (ES)", "Chinese (ZH)", "Korean (KO)", "Russian (RU)", "Arabic (AR)", "Japanese (JA)"]

if translation_method == "هوش مصنوعی":
    # @markdown  **لطفاً کلید خود را در کادر زیر وارد کنید (API KEY)**
    GOOGLE_API_KEY = "" #@param {type:"string"}

    if not GOOGLE_API_KEY:
        print("هشدار: کلید API وارد نشده است.")
    else:
        genai.configure(api_key=GOOGLE_API_KEY)
        print("کلید API با موفقیت پیکربندی شد.")

        filename = '/content/audio.srt'
        output_filename = '/content/audio_translated.srt'

        @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
        def translate_subtitle_with_retry(text):
            model = genai.GenerativeModel('gemini-1.5-flash', safety_settings={
                genai.types.HarmCategory.HARM_CATEGORY_HARASSMENT: genai.types.HarmBlockThreshold.BLOCK_NONE,
                genai.types.HarmCategory.HARM_CATEGORY_HATE_SPEECH: genai.types.HarmBlockThreshold.BLOCK_NONE,
                genai.types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: genai.types.HarmBlockThreshold.BLOCK_NONE,
                genai.types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: genai.types.HarmBlockThreshold.BLOCK_NONE,
            })

            # تنظیم پرامپت براساس زبان مقصد
            language_map = {
                "Persian (FA)": "فارسی",
                "English (EN)": "English",
                "German (DE)": "German",
                "French (FR)": "French",
                "Italian (IT)": "Italian",
                "Spanish (ES)": "Spanish",
                "Chinese (ZH)": "Chinese",
                "Korean (KO)": "Korean",
                "Russian (RU)": "Russian",
                "Arabic (AR)": "Arabic",
                "Japanese (JA)": "Japanese"
            }

            target_lang_name = language_map.get(target_language, "English")

            if target_language == "Persian (FA)":
                prompt = f"""دستورالعمل:
                1. فقط متن را به فارسی عامیانه و لحن خودمونی ترجمه کن
                2. هرجا لازمه از نقطه و کاما و علائم نگارشی استفاده کن
                3. اضافه گویی در ترجمه ممنوع
                متن برای ترجمه:
                {text}"""
            else:
                prompt = f"""Instruction:
                1. Please translate the text to {target_lang_name} with the same tone
                2. Use appropriate punctuation where necessary
                3. No additional explanation or text
                Text to translate:
                {text}"""

            response = model.generate_content(prompt)
            time.sleep(3)
            return response.text

        try:
            subs = pysrt.open(filename)
            with tqdm(total=len(subs), desc="ترجمه زیرنویس") as pbar:
                for i, sub in enumerate(subs):
                    sub.text = translate_subtitle_with_retry(sub.text)
                    pbar.update(1)
            subs.save(output_filename, encoding='utf-8')
            os.rename(output_filename, 'audio_fa.srt')  # برای سازگاری با کد قبلی
            print(f"\nترجمه از {source_language} به {target_language} با موفقیت به پایان رسید!")
        except Exception as e:
            print(f"خطا در خواندن فایل زیرنویس: {str(e)}")

else:
    print("لطفاً فایل زیرنویس ترجمه شده خود را آپلود کنید:")
    uploaded = files.upload()
    subtitle_file = next(iter(uploaded.keys()))
    os.rename(subtitle_file, 'audio_fa.srt')
    print("زیرنویس ترجمه شده با موفقیت آپلود شد.")

In [ ]:
#@title ساخت سگمنت‌های صوتی با زمان‌بندی دقیق (بهبود یافته)
import edge_tts
import asyncio
import pysrt
import os
import subprocess
import traceback
from pydub import AudioSegment

os.makedirs('dubbing_project/dubbed_segments', exist_ok=True)

# استخراج کد زبان از voice_choice برای نام فایل
voice_choice = "فرید (FA)" #@param ["فرید (FA)", "دلارا (FA)", "Jenny (EN)", "Guy (EN)", "Katja (DE)", "Conrad (DE)", "Elvira (ES)", "Alvaro (ES)", "Denise (FR)", "Henri (FR)", "Nanami (JA)", "Keita (JA)", "SunHi (KO)", "InJoon (KO)", "Xiaoxiao (ZH)", "Yunyang (ZH)", "Svetlana (RU)", "Dmitry (RU)", "Amina (AR)", "Hamed (AR)", "Isabella (IT)", "Diego (IT)"]
voice_code = voice_choice.split("(")[1].split(")")[0] if "(" in voice_choice else "FA"
output_filename = f'final_dubbed_video_{voice_code}.mp4'

# تعریف گوینده‌ها برای همه زبان‌ها
VOICE_MAP = {
    # فارسی
    "فرید (FA)": "fa-IR-FaridNeural",
    "دلارا (FA)": "fa-IR-DilaraNeural",
    # انگلیسی
    "Jenny (EN)": "en-US-JennyNeural",
    "Guy (EN)": "en-US-GuyNeural",
    # آلمانی
    "Katja (DE)": "de-DE-KatjaNeural",
    "Conrad (DE)": "de-DE-ConradNeural",
    # فرانسوی
    "Denise (FR)": "fr-FR-DeniseNeural",
    "Henri (FR)": "fr-FR-HenriNeural",
    # ایتالیایی
    "Isabella (IT)": "it-IT-IsabellaNeural",
    "Diego (IT)": "it-IT-DiegoNeural",
    # اسپانیایی
    "Elvira (ES)": "es-ES-ElviraNeural",
    "Alvaro (ES)": "es-ES-AlvaroNeural",
    # چینی
    "Xiaoxiao (ZH)": "zh-CN-XiaoxiaoNeural",
    "Yunyang (ZH)": "zh-CN-YunyangNeural",
    # کره‌ای
    "SunHi (KO)": "ko-KR-SunHiNeural",
    "InJoon (KO)": "ko-KR-InJoonNeural",
    # روسی
    "Svetlana (RU)": "ru-RU-SvetlanaNeural",
    "Dmitry (RU)": "ru-RU-DmitryNeural",
    # عربی
    "Amina (AR)": "ar-EG-AminaNeural",
    "Hamed (AR)": "ar-EG-HamedNeural",
    # ژاپنی
    "Nanami (JA)": "ja-JP-NanamiNeural",
    "Keita (JA)": "ja-JP-KeitaNeural"
}

async def generate_speech():
    # نصب rubberband-cli اگر نصب نیست
    try:
        subprocess.run(['apt-get', 'install', '-y', 'rubberband-cli'], check=True)
        print("rubberband-cli با موفقیت نصب شد.")
    except Exception as e:
        print(f"خطا در نصب rubberband-cli: {str(e)}")
        print("در حال ادامه با روش جایگزین...")

    try:
        subs = pysrt.open('/content/audio_fa.srt', encoding='utf-8')
    except UnicodeDecodeError:
        # اگر utf-8 نبود، سعی کنید با latin-1
        subs = pysrt.open('/content/audio_fa.srt', encoding='latin-1')
    except Exception as e:
        print(f"خطا در خواندن فایل زیرنویس: {str(e)}")
        print("در حال تلاش با فرمت دیگر...")
        try:
            # تبدیل فایل زیرنویس به utf-8
            subprocess.run(['iconv', '-f', 'ISO-8859-1', '-t', 'UTF-8', '/content/audio_fa.srt', '-o', '/content/audio_fa_utf8.srt'])
            subs = pysrt.open('/content/audio_fa_utf8.srt')
            print("فایل زیرنویس با موفقیت به UTF-8 تبدیل شد.")
        except Exception as e2:
            print(f"باز هم خطا در خواندن فایل زیرنویس: {str(e2)}")
            return

    selected_voice = VOICE_MAP.get(voice_choice)
    if not selected_voice:
        print(f"گوینده انتخاب شده '{voice_choice}' در لیست موجود نیست. لطفاً یک گوینده معتبر انتخاب کنید.")
        return

    print(f"شروع تولید سگمنت‌های صوتی با صدای {voice_choice} ({selected_voice})...")

    # ایجاد فایل log برای ثبت وضعیت سگمنت‌ها
    with open('dubbing_project/segments_log.txt', 'w', encoding='utf-8') as log_file:
        log_file.write("=== گزارش وضعیت سگمنت‌های صوتی ===\n\n")

        for i, sub in enumerate(subs):
            try:
                # ثبت اطلاعات زیرنویس در لاگ
                log_file.write(f"سگمنت {i+1}:\n")
                log_file.write(f"  متن: {sub.text}\n")
                log_file.write(f"  زمان شروع: {sub.start}\n")
                log_file.write(f"  زمان پایان: {sub.end}\n")

                # محاسبه مدت زمان دقیق
                start_time = sub.start.seconds + sub.start.minutes * 60 + sub.start.hours * 3600 + sub.start.milliseconds/1000
                end_time = sub.end.seconds + sub.end.minutes * 60 + sub.end.hours * 3600 + sub.end.milliseconds/1000
                target_duration = end_time - start_time

                # اگر مدت زمان صفر یا منفی باشد، اصلاح شود
                if target_duration <= 0:
                    log_file.write(f"  هشدار: مدت زمان نامعتبر ({target_duration}s)، تنظیم به 1 ثانیه\n")
                    target_duration = 1.0

                log_file.write(f"  مدت زمان هدف: {target_duration:.2f}s\n")

                # تولید صدا با Edge TTS
                communicate = edge_tts.Communicate(sub.text, selected_voice)
                await communicate.save(f"dubbing_project/dubbed_segments/temp_{i+1}.mp3")
                log_file.write(f"  فایل MP3 اولیه ایجاد شد\n")

                # استفاده از ffmpeg برای تبدیل به wav
                try:
                    subprocess.run([
                        'ffmpeg', '-i', f"dubbing_project/dubbed_segments/temp_{i+1}.mp3",
                        '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2',
                        '-y', f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav"
                    ], check=True)
                    log_file.write(f"  تبدیل به WAV با موفقیت انجام شد\n")
                except subprocess.CalledProcessError:
                    log_file.write(f"  خطا در تبدیل به WAV، در حال استفاده از pydub\n")
                    # استفاده از pydub به عنوان جایگزین
                    sound = AudioSegment.from_mp3(f"dubbing_project/dubbed_segments/temp_{i+1}.mp3")
                    sound.export(f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav", format="wav")
                    log_file.write(f"  تبدیل به WAV با pydub با موفقیت انجام شد\n")

                # محاسبه مدت زمان اصلی فایل صوتی
                try:
                    result = subprocess.run([
                        'ffprobe', '-v', 'error', '-show_entries', 'format=duration',
                        '-of', 'default=noprint_wrappers=1:nokey=1',
                        f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav"
                    ], capture_output=True, text=True, check=True)
                    original_duration = float(result.stdout.strip())
                    log_file.write(f"  مدت زمان اصلی: {original_duration:.2f}s\n")
                except:
                    # اگر ffprobe کار نکرد، از pydub استفاده کن
                    sound = AudioSegment.from_file(f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav")
                    original_duration = len(sound) / 1000.0  # تبدیل میلی‌ثانیه به ثانیه
                    log_file.write(f"  مدت زمان اصلی (با pydub): {original_duration:.2f}s\n")

                # محاسبه ضریب سرعت
                speed_factor = original_duration / target_duration

                # محدود کردن ضریب سرعت برای جلوگیری از خطا
                if speed_factor < 0.5:
                    speed_factor = 0.5
                    log_file.write(f"  هشدار: ضریب سرعت خیلی کم، تنظیم به 0.5\n")
                elif speed_factor > 2.0:
                    speed_factor = 2.0
                    log_file.write(f"  هشدار: ضریب سرعت خیلی زیاد، تنظیم به 2.0\n")

                log_file.write(f"  ضریب سرعت نهایی: {speed_factor}\n")

                # روش 1: استفاده از rubberband (اگر نصب باشد)
                try:
                    subprocess.run([
                        'ffmpeg', '-i', f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav",
                        '-filter:a', f'rubberband=tempo={speed_factor}',
                        '-y', f"dubbing_project/dubbed_segments/dub_{i+1}.wav"
                    ], check=True)
                    log_file.write(f"  تغییر سرعت با rubberband با موفقیت انجام شد\n")
                except:
                    # روش 2: استفاده از فیلتر atempo فقط در محدوده 0.5 تا 2.0
                    log_file.write(f"  خطا در استفاده از rubberband، در حال استفاده از atempo\n")
                    try:
                        subprocess.run([
                            'ffmpeg', '-i', f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav",
                            '-filter:a', f'atempo={speed_factor}',
                            '-y', f"dubbing_project/dubbed_segments/dub_{i+1}.wav"
                        ], check=True)
                        log_file.write(f"  تغییر سرعت با atempo با موفقیت انجام شد\n")
                    except:
                        # روش 3: استفاده از pydub
                        log_file.write(f"  خطا در استفاده از atempo، در حال استفاده از pydub\n")
                        try:
                            # کپی فایل اصلی بدون تغییر سرعت
                            subprocess.run([
                                'cp', f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav",
                                f"dubbing_project/dubbed_segments/dub_{i+1}.wav"
                            ], check=True)
                            log_file.write(f"  استفاده از فایل اصلی بدون تغییر سرعت\n")
                        except:
                            log_file.write(f"  همه روش‌ها با خطا مواجه شدند\n")
                            raise Exception("خطا در ایجاد فایل صوتی")

                # اطمینان از وجود فایل نهایی
                if not os.path.exists(f"dubbing_project/dubbed_segments/dub_{i+1}.wav"):
                    log_file.write(f"  خطا: فایل نهایی ایجاد نشد!\n")
                    # ایجاد یک فایل صدای خالی به عنوان پشتیبان
                    silent_audio = AudioSegment.silent(duration=int(target_duration * 1000))
                    silent_audio.export(f"dubbing_project/dubbed_segments/dub_{i+1}.wav", format="wav")
                    log_file.write(f"  یک فایل سکوت به عنوان جایگزین ایجاد شد\n")
                else:
                    # بررسی مدت زمان فایل نهایی
                    try:
                        result = subprocess.run([
                            'ffprobe', '-v', 'error', '-show_entries', 'format=duration',
                            '-of', 'default=noprint_wrappers=1:nokey=1',
                            f"dubbing_project/dubbed_segments/dub_{i+1}.wav"
                        ], capture_output=True, text=True)
                        final_duration = float(result.stdout.strip())
                        log_file.write(f"  مدت زمان نهایی: {final_duration:.2f}s\n")
                    except:
                        log_file.write(f"  نمی‌توان مدت زمان فایل نهایی را تعیین کرد\n")

                # پاکسازی فایل‌های موقت
                if os.path.exists(f"dubbing_project/dubbed_segments/temp_{i+1}.mp3"):
                    os.remove(f"dubbing_project/dubbed_segments/temp_{i+1}.mp3")
                if os.path.exists(f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav"):
                    os.remove(f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav")

                log_file.write(f"  بخش {i+1} با موفقیت ساخته شد\n\n")
                print(f"بخش {i+1} از {len(subs)} ساخته شد")

            except Exception as e:
                # در صورت خطا، ایجاد یک فایل صدای خالی
                log_file.write(f"  خطای کلی: {str(e)}\n")
                log_file.write(f"  جزئیات خطا: {traceback.format_exc()}\n")
                log_file.write(f"  در حال ایجاد فایل سکوت به عنوان جایگزین\n")

                try:
                    # محاسبه مدت زمان دقیق
                    start_time = sub.start.seconds + sub.start.minutes * 60 + sub.start.hours * 3600 + sub.start.milliseconds/1000
                    end_time = sub.end.seconds + sub.end.minutes * 60 + sub.end.hours * 3600 + sub.end.milliseconds/1000
                    target_duration = max(end_time - start_time, 0.5)  # حداقل 0.5 ثانیه

                    # ایجاد فایل سکوت
                    silent_audio = AudioSegment.silent(duration=int(target_duration * 1000))
                    silent_audio.export(f"dubbing_project/dubbed_segments/dub_{i+1}.wav", format="wav")
                    log_file.write(f"  فایل سکوت با مدت زمان {target_duration:.2f}s ایجاد شد\n\n")
                except Exception as e2:
                    log_file.write(f"  خطا در ایجاد فایل سکوت: {str(e2)}\n\n")
                    print(f"خطا در پردازش بخش {i+1}: {str(e)}")

    print(f"تمام سگمنت‌های صوتی با صدای {voice_choice} ساخته شدند!")
    print(f"برای اطلاعات بیشتر، فایل گزارش در dubbing_project/segments_log.txt را بررسی کنید.")

# نصب rubberband-cli اگر نصب نیست
!apt-get install -y rubberband-cli

await generate_speech()

In [ ]:
#@title شروع عملیات نهایی (بهبود یافته)
import subprocess
import pysrt
import os
from datetime import timedelta
import json
from pydub import AudioSegment
import tempfile
import glob

# بررسی و پیدا کردن فایل ویدیو با هر فرمتی
video_files = glob.glob('input_video.*')
if not os.path.exists('input_video.mp4') and video_files:
    input_video = video_files[0]
else:
    input_video = 'input_video.mp4'

#@markdown #### تنظیمات صدای اصلی
#@markdown آیا می‌خواهید صدای اصلی ویدیو حفظ شود؟
keep_original_audio = False #@param {type:"boolean"}
#@markdown میزان صدای اصلی ویدیو (فقط در صورت فعال بودن گزینه بالا)
original_audio_volume = 0.05 #@param {type:"slider", min:0, max:1, step:0.005}
#@markdown #### روش ترکیب صدا
audio_merge_method = "ffmpeg concat" #@param ["ffmpeg concat", "ffmpeg filter_complex", "pydub"]

# بررسی وجود فایل‌های مورد نیاز
if not os.path.exists(input_video):
    print(f"خطا: فایل {input_video} یافت نشد!")
    print("لطفاً نام ویدیوی ورودی را بررسی کنید")
else:
    print(f"✓ فایل ویدیو یافت شد: {input_video}")

if not os.path.exists('/content/audio_fa.srt'):
    print("خطا: فایل زیرنویس یافت نشد!")
else:
    print("✓ فایل زیرنویس یافت شد")

# بررسی وجود پوشه سگمنت‌ها
segment_dir = "dubbing_project/dubbed_segments"
if not os.path.exists(segment_dir):
    print("خطا: پوشه سگمنت‌های صوتی یافت نشد!")
    os.makedirs(segment_dir, exist_ok=True)
else:
    segments = [f for f in os.listdir(segment_dir) if f.startswith("dub_") and f.endswith(".wav")]
    print(f"✓ {len(segments)} سگمنت صوتی یافت شد")

# خواندن و پردازش فایل زیرنویس
try:
    subs = pysrt.open('/content/audio_fa.srt', encoding='utf-8')
    print("✓ فایل زیرنویس با موفقیت خوانده شد")
except UnicodeDecodeError:
    try:
        subs = pysrt.open('/content/audio_fa.srt', encoding='latin-1')
        print("✓ فایل زیرنویس با encoding=latin-1 خوانده شد")
    except Exception as e:
        print(f"خطا در خواندن فایل زیرنویس: {str(e)}")
        print("در حال تلاش مجدد...")
        try:
            subprocess.run(['iconv', '-f', 'ISO-8859-1', '-t', 'UTF-8', '/content/audio_fa.srt', '-o', '/content/audio_fa_utf8.srt'])
            subs = pysrt.open('/content/audio_fa_utf8.srt')
            print("✓ فایل زیرنویس با تبدیل به UTF-8 خوانده شد")
        except:
            print("خطا در خواندن فایل زیرنویس!")
            exit()

# استخراج کد زبان از voice_choice برای نام فایل
voice_choice = os.environ.get('voice_choice', "دلارا (FA)")
voice_code = voice_choice.split("(")[1].split(")")[0] if "(" in voice_choice else "FA"
output_filename = f'final_dubbed_video_{voice_code}.mp4'

# برای اطمینان از پاک شدن فایل خروجی قبلی
if os.path.exists(output_filename):
    os.remove(output_filename)
    print(f"✓ فایل خروجی قبلی '{output_filename}' حذف شد")

# روش 1: استفاده از فایل concat
if audio_merge_method == "ffmpeg concat":
    print("در حال استفاده از روش ffmpeg concat...")

    try:
        temp_dir = tempfile.mkdtemp()
        segment_info_file = os.path.join(temp_dir, "segments.txt")

        # استخراج صدای اصلی
        original_audio_path = os.path.join(temp_dir, "original_audio.wav")
        subprocess.run([
            'ffmpeg', '-i', input_video, '-vn',
            '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2',
            original_audio_path
        ], check=True)

        # تنظیم میزان صدای اصلی
        modified_original_audio = os.path.join(temp_dir, "original_audio_modified.wav")
        volume = original_audio_volume if keep_original_audio else 0.0
        subprocess.run([
            'ffmpeg', '-i', original_audio_path,
            '-filter:a', f'volume={volume}',
            modified_original_audio
        ], check=True)

        # ایجاد فایل خروجی با آدیو ترکیب شده
        with open(segment_info_file, 'w') as f:
            for i, sub in enumerate(subs):
                start_time_ms = (sub.start.hours * 3600 + sub.start.minutes * 60 + sub.start.seconds) * 1000 + sub.start.milliseconds
                start_time_sec = start_time_ms / 1000.0

                segment_path = f"dubbing_project/dubbed_segments/dub_{i+1}.wav"
                if os.path.exists(segment_path):
                    f.write(f"file '{segment_path}'\n")
                    f.write(f"inpoint {start_time_sec}\n")

        # ترکیب سگمنت‌ها با صدای اصلی
        merged_audio = os.path.join(temp_dir, "merged_audio.wav")
        dubbing_audio = os.path.join(temp_dir, "dubbing_audio.wav")

        subprocess.run([
            'ffmpeg', '-f', 'concat', '-safe', '0',
            '-i', segment_info_file, '-c', 'copy', dubbing_audio
        ], check=True)

        subprocess.run([
            'ffmpeg',
            '-i', modified_original_audio,
            '-i', dubbing_audio,
            '-filter_complex', '[0:a][1:a]amix=inputs=2:duration=longest',
            merged_audio
        ], check=True)

        subprocess.run([
            'ffmpeg', '-i', input_video, '-i', merged_audio,
            '-c:v', 'copy', '-c:a', 'aac', '-map', '0:v', '-map', '1:a',
            output_filename
        ], check=True)

        print(f"✓ ویدیو با موفقیت با روش concat ساخته شد: {output_filename}")
        from google.colab import files
        files.download(output_filename)

    except Exception as e:
        print(f"خطا در روش concat: {str(e)}")
        print("در حال امتحان روش جایگزین...")
        audio_merge_method = "pydub"

# روش 2: استفاده از filter_complex
if audio_merge_method == "ffmpeg filter_complex":
    print("در حال استفاده از روش ffmpeg filter_complex...")

    try:
        if keep_original_audio:
            filter_complex = f"[0:a]volume={original_audio_volume}[original_audio];"
        else:
            filter_complex = "[0:a]volume=0[original_audio];"

        valid_segments = []
        for i, sub in enumerate(subs):
            segment_path = f"dubbing_project/dubbed_segments/dub_{i+1}.wav"
            if os.path.exists(segment_path):
                try:
                    start_time_ms = (sub.start.hours * 3600 + sub.start.minutes * 60 + sub.start.seconds) * 1000 + sub.start.milliseconds
                    if start_time_ms < 0:
                        start_time_ms = 0
                    filter_complex += f"[{i+1}:a]adelay={start_time_ms}|{start_time_ms}[a{i+1}];"
                    valid_segments.append(i)
                except Exception as e:
                    print(f"رد کردن سگمنت {i+1} به دلیل مشکل: {str(e)}")
                    continue

        if valid_segments:
            merge_command = "[original_audio]"
            for i in valid_segments:
                merge_command += f"[a{i+1}]"
            merge_command += f"amix=inputs={len(valid_segments) + 1}:normalize=0[aout]"
            filter_complex += merge_command

            input_files = f"-i {input_video} " + " ".join([f"-i dubbing_project/dubbed_segments/dub_{i+1}.wav" for i in valid_segments])
            command = f"ffmpeg -y {input_files} -filter_complex \"{filter_complex}\" -map 0:v -map \"[aout]\" -c:v copy -c:a aac {output_filename}"

            result = subprocess.run(command, shell=True, capture_output=True, text=True)
            print("خروجی ffmpeg (filter_complex):")
            print(result.stdout)
            print("خطاهای ffmpeg (filter_complex):")
            print(result.stderr)

            if os.path.exists(output_filename):
                print(f"✓ ویدیو با موفقیت با روش filter_complex ساخته شد: {output_filename}")
                from google.colab import files
                files.download(output_filename)
            else:
                raise Exception("فایل خروجی ساخته نشد!")
        else:
            print("خطا: هیچ سگمنت صوتی معتبری یافت نشد!")
            audio_merge_method = "pydub"

    except Exception as e:
        print(f"خطا در روش filter_complex: {str(e)}")
        print("در حال امتحان روش جایگزین...")
        audio_merge_method = "pydub"

# روش 3: استفاده از pydub
if audio_merge_method == "pydub":
    print("در حال استفاده از روش pydub...")

    try:
        temp_dir = tempfile.mkdtemp()
        original_audio_path = os.path.join(temp_dir, "original_audio.wav")
        subprocess.run([
            'ffmpeg', '-i', input_video, '-vn',
            '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2',
            original_audio_path
        ], check=True)

        original_audio = AudioSegment.from_file(original_audio_path)
        if keep_original_audio:
            original_audio = original_audio - (20 - (20 * original_audio_volume))
        else:
            original_audio = AudioSegment.silent(duration=len(original_audio))

        final_audio = original_audio
        for i, sub in enumerate(subs):
            segment_path = f"dubbing_project/dubbed_segments/dub_{i+1}.wav"
            if os.path.exists(segment_path):
                try:
                    segment_audio = AudioSegment.from_file(segment_path)
                    start_time_ms = (sub.start.hours * 3600 + sub.start.minutes * 60 + sub.start.seconds) * 1000 + sub.start.milliseconds
                    if start_time_ms < 0:
                        start_time_ms = 0
                    final_audio = final_audio.overlay(segment_audio, position=start_time_ms)
                    print(f"سگمنت {i+1} در زمان {start_time_ms}ms اضافه شد")
                except Exception as e:
                    print(f"خطا در اضافه کردن سگمنت {i+1}: {str(e)}")

        merged_audio_path = os.path.join(temp_dir, "merged_audio.wav")
        final_audio.export(merged_audio_path, format="wav")

        subprocess.run([
            'ffmpeg', '-i', input_video, '-i', merged_audio_path,
            '-c:v', 'copy', '-c:a', 'aac', '-map', '0:v', '-map', '1:a',
            output_filename
        ], check=True)

        print(f"✓ ویدیو با موفقیت با روش pydub ساخته شد: {output_filename}")
        from google.colab import files
        files.download(output_filename)

    except Exception as e:
        print(f"خطا در روش pydub: {str(e)}")
        print("لطفاً خطا را بررسی کنید، هیچ روش دیگری باقی نمانده است!")

# پاکسازی فایل‌های موقت
if os.path.exists(temp_dir):
    import shutil
    shutil.rmtree(temp_dir)
    print("✓ فایل‌های موقت پاک شدند")

In [ ]:
#@title پاکسازی فایل‌های جلسه قبلی
!rm -rf /content/*